In [1]:
import tensorflow as tf 
import os 

/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Change directory
os.chdir("/home/saurabh/Downloads/CatDog") 

In [3]:
#Get working directory
os.getcwd() 

'/home/saurabh/Downloads/CatDog'

In [4]:
#Define a list containing filenames 
image_paths = []

for names in os.listdir("/home/saurabh/Downloads/CatDog"):
    image_paths.append(names) 

In [5]:
#Retreiving filenames 
#Commented as it has long ouput 
'''image_paths''' 


'image_paths'

In [6]:
#No. of images 
len(image_paths) 

25000

In [7]:
#Creating tensor object for images 
data_images = tf.constant(image_paths)  

In [8]:
#Obtain the labels corresponding to each image 
labels = []

for names in os.listdir("/home/saurabh/Downloads/CatDog"):
    
    if names.startswith("c"):
        label = 0
    elif names.startswith("d"):
        label = 1
    labels.append(label)
                      

In [9]:
len(labels)

25000

In [10]:
#Creating tensor object for labels
data_labels = tf.constant(labels)   

In [11]:
#Creating train and validation dataset from tensor objects
tr_data = tf.data.Dataset.from_tensor_slices((data_images[:20000], data_labels[:20000]))  
val_data = tf.data.Dataset.from_tensor_slices((data_images[20000:], data_labels[20000:])) 

In [12]:
#Creating a generic iterator object 
iterator = tf.data.Iterator.from_structure(tr_data.output_types,tr_data.output_shapes) 
 
#Accessing the next element from the iterator 
next_element = iterator.get_next()  

In [13]:
#Initializing iterator for train and validation set
train_init_op = iterator.make_initializer(tr_data) 
val_init_op = iterator.make_initializer(val_data) 

In [14]:
#Retreiving created data with images and label
#Commented as it has long ouput 

'''with tf.Session() as sess:
    sess.run(train_init_op)
    while True:
        try:
            elem = sess.run(next_element)
            print(elem)
             
        except tf.errors.OutOfRangeError: 
            print("End of training dataset.")
            break '''

'with tf.Session() as sess:\n    sess.run(train_init_op)\n    while True:\n        try:\n            elem = sess.run(next_element)\n            print(elem)\n             \n        except tf.errors.OutOfRangeError: \n            print("End of training dataset.")\n            break '

In [15]:
#Defining an input parser
def parse_function(filename, label):
    image_string = tf.read_file(filename)
    

    image = tf.image.decode_jpeg(image_string, channels=3)
      
    # Converting to float values 
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_images(image, [64, 64]) 
    return image, label 

In [16]:
tr_data = tr_data.map(parse_function) 

In [17]:
val_data = val_data.map(parse_function) 

In [18]:
#Defining augmentation functions 
def img_augmentation(image, label):
    
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, 0.2, 1.8, seed=1) 
    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_hue(image, max_delta=32.0 / 255) 
   
    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label  

In [19]:
#Augmentated Data 
Aug_data = tr_data.map(img_augmentation, num_parallel_calls=4)   

In [20]:
#Concatenating augmentated data with the trainset 
train_data = tr_data.concatenate(Aug_data) 

In [21]:
#Shuffle the trainset
train_data = train_data.shuffle(len(image_paths)) 

In [22]:
#Batch size 
train_data = train_data.batch(16) 
train_data = train_data.prefetch(1)  
val_data = val_data.batch(16) 
val_data = val_data.prefetch(1) 

In [23]:
iterator = train_data.make_initializable_iterator() 
 
#Accessing the next element from the iterator 
images, labels = iterator.get_next()   

In [24]:
#Image shape
images 

<tf.Tensor 'IteratorGetNext_1:0' shape=(?, 64, 64, 3) dtype=float32>

In [25]:
#Labels shape 
labels 

<tf.Tensor 'IteratorGetNext_1:1' shape=(?,) dtype=int32>

In [26]:
#Train and validation set iterator initialization 
training_init_op = iterator.make_initializer(train_data)   
valid_init_op = iterator.make_initializer(val_data) 

In [27]:
#Retreiving image arrays after parsing 
#Commented as it has long ouput 

'''with tf.Session() as sess:
    sess.run(training_init_op) 
    for i in range(5):
        elem = sess.run(images) 
        print(elem)'''


'with tf.Session() as sess:\n    sess.run(training_init_op) \n    for i in range(5):\n        elem = sess.run(images) \n        print(elem)'

In [28]:
#Stacking layers as per VGG first block 
conv1 = tf.layers.conv2d(images, filters= 64, kernel_size= (3,3), strides=(1,1), activation=tf.nn.relu, kernel_initializer= tf.truncated_normal_initializer)  
conv2 = tf.layers.conv2d(conv1, filters= 64, kernel_size= (3,3), strides=(1,1), activation=tf.nn.relu)   
max1 = tf.layers.max_pooling2d(conv2, pool_size=(2,2), strides = (2,2)) 
flatten = tf.layers.flatten(max1) 

#Adding fully connected layers 
fc1 = tf.layers.dense(flatten, 512, activation= tf.nn.sigmoid, kernel_initializer=tf.truncated_normal_initializer) 
fc2 = tf.layers.dense(fc1, 2)    


In [29]:
loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= labels, logits=fc2)) 
optimizer = tf.train.AdamOptimizer().minimize(loss)
# get accuracy
prediction = tf.argmax(fc2, 1, output_type=tf.int32)
equality = tf.equal(prediction, labels) 
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))    
init_op = tf.global_variables_initializer()   

In [30]:
epochs = 100
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
         
        l, _, acc = sess.run([loss, optimizer, accuracy])
        if i % 5== 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100)) 
            
    valid_iters= 200
    sess.run(valid_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        acc = sess.run([accuracy]) 
        avg_acc += acc[0] 
    print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters, (avg_acc / valid_iters) * 100))

            
            
    

Epoch: 0, loss: 12.746, training accuracy: 31.25%
Epoch: 5, loss: 11.683, training accuracy: 56.25%
Epoch: 10, loss: 13.973, training accuracy: 37.50%
Epoch: 15, loss: 10.829, training accuracy: 68.75%
Epoch: 20, loss: 10.679, training accuracy: 68.75%
Epoch: 25, loss: 10.833, training accuracy: 50.00%
Epoch: 30, loss: 11.874, training accuracy: 56.25%
Epoch: 35, loss: 11.877, training accuracy: 43.75%
Epoch: 40, loss: 12.370, training accuracy: 43.75%
Epoch: 45, loss: 12.052, training accuracy: 37.50%
Epoch: 50, loss: 16.115, training accuracy: 25.00%
Epoch: 55, loss: 11.513, training accuracy: 43.75%
Epoch: 60, loss: 12.758, training accuracy: 18.75%
Epoch: 65, loss: 11.700, training accuracy: 43.75%
Epoch: 70, loss: 11.385, training accuracy: 37.50%
Epoch: 75, loss: 10.437, training accuracy: 62.50%
Epoch: 80, loss: 9.796, training accuracy: 75.00%
Epoch: 85, loss: 12.275, training accuracy: 43.75%
Epoch: 90, loss: 10.933, training accuracy: 56.25%
Epoch: 95, loss: 10.580, training 